# Chapter 1. The Python Data Model
- 파이썬의 가장 큰 장점 중 하나는 **일관성**: 일관성 있는 문법 덕분에 파이썬을 배운지 얼마되지 않은 사람들도 쉽게 프로그램을 작성할 수 있음
- 그러나 파이썬 이전에 다른 **객체 지향 언어**를 배운 적이 있다면, `collection.len()` 대신 `len(collection)`을 사용하는 파이썬이 어색할 수 있음
- 이러한 차이가 우리가 흔히 이야기하는 **파이써닉**의 핵심 개념
- 우리는 **파이써닉**의 개념을 통해 구현한 객체를 **파이썬 데이터 모델**이라 부르며, 이를 통해 사용자가 객체를 쉽게 사용할 수 있는 **API** 구현 가능
- 즉, 데이터 모델을 통해 시퀀스, 이터레이터, 함수, 클래스, 컨텍스트 매니저 등의 **인터페이스**를 형식화 할 수 있는 것
- 이제 파이썬의 **스페셜 메서드**를 통해 다양한 기본 연산을 수행해보자

## 1.1 파이써닉한 카드 덱
- 본 예제는 매우 간단하지만, `__getitem__`과 `__len__`이라는 **스페셜 메서드**를 구현하는 것이 얼마나 강력한지를 보여주는 예제

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

- 처음으로 눈여겨 봐야 할 것은 **collections.namedtuple**을 활용해 개별 카드를 나타내는 간단한 클래스를 구현한 것
- 파이썬 2.6 이후로 **namedtuple**은 커스텀 메서드가 없는 간단한 클래스를 생성하기 위해 유용하게 사용되고 있음

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

`FrenchDeck` 클래스는 파이썬의 다른 자료구조들과 마찬가지로 포함하고 있는 **원소의 개수를 반환**하는 `len()` 함수를 지원

In [3]:
deck = FrenchDeck()
len(deck)

52

또한 `__getitem__` 메서드를 통해 **특정 인덱스**에 해당하는 원소를 반환해 볼 수도 있음

In [4]:
deck[0]

Card(rank='2', suit='spades')

In [5]:
deck[-1]

Card(rank='A', suit='hearts')

파이썬은 `random.choice` 라는 함수를 지원하기 때문에 임의의 카드를 뽑기 위해 굳이 새로운 메서드를 정의할 필요는 없음

In [6]:
from random import choice

choice(deck)

Card(rank='10', suit='clubs')

파이썬 데이터 모델을 정의할 때, **스페셜 메서드**를 활용하는 것이 좋은 이유:
- 사용자가 원소의 개수를 확인하기 위한 함수명이 무엇인지 따로 기억하지 않아도 된다는 점
    - 원소의 개수를 반환하는 함수를 누구는 `get_length` 누구는 `count_length`와 같이 작성할 수 있는데, 이로 인해 발생하는 혼란을 줄일 수 있음
- 바퀴를 새로 만들 필요 없이 파이썬의 **정규 라이브러리**를 활용할 수 있음 (e.g. `random.choice`)
- 리스트의 `[ ]` 연산자를 활용해 `__getitem__`을 정의했기 때문에 **슬라이싱** 연산의 혜택도 얻을 수 있음

In [7]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [8]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

`__getitem__`이라는 스페셜 메서드 덕분에 `FrenchDeck`은 **이터러블**하기도 함

In [ ]:
for card in deck:
    print(card)

**이터러블**하기 때문에 `reversed` 형태로 순회도 가능

In [ ]:
for card in reversed(deck):
    print(card)

- 순회는 때로 **암묵적으로** 적용되기도 함
- 만약 collection이 `__contains__` 메서드를 지니지 않는 경우, `in` 연산자는 **시퀀셜 스캔**을 수행
- `FrenchDeck`은 이터러블하기 때문에 `in` 연산에서 순회가 암묵적으로 작동

In [9]:
Card('Q', 'hearts') in deck

True

In [10]:
Card('7', 'beasts') in deck

False

카드를 정렬하는 보편적 방법은 **랭크**로 먼저 정렬한 후, **문양**으로 정렬하는 것

In [11]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
suit_values

{'spades': 3, 'hearts': 2, 'diamonds': 1, 'clubs': 0}

In [12]:
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

아래 코드에서는 `spades_high`라는 함수가 먼저 실행되 각 데이터 인스턴스들에 **키 값**을 부여하게 되고, 해당 키 값을 기준으로 정렬이 수행됨

In [ ]:
for card in sorted(deck, key=spades_high):
    print(card)

`__len__` 메서드와 `__getitem__` 메서드를 정의함으로써 `FrenchDeck`을 파이썬의 **정규 시퀀스**와 같이 사용할 수 있게 되었음: **슬라이싱**은 덤

## 1.2 스페셜 메서드는 어떻게 사용되는가
- **스페셜 메서드**는 사용자가 아닌 **파이썬 인터프리터가 호출**하는 함수
- 사용자는 `obj.__len__()` 대신 `len(obj)`로 **스페셜 메서드**를 호출
- 그러나 `list`, `str`, `bytearray와` 같은 빌트인 타입들의 경우 `len()` 함수가 실제로 **CPython**에 의해 **PyVarObject C** 구조체에 있던 객체의 크기 반환
- **스페셜 메서드**는 대개 **암시적으로** 호출되는데, `for i in x` 역시 사실 `x.__iter__()`의 **스페셜 메서드**를 **암시적으로** 호출하는 구조
- **스페셜 메서드**를 호출하는 경우 `len()`, `iter()`, `str()`과 같이 관련 **내장 함수**를 호출하는 것이 좋음: 해당 내장 함수들이 **스페셜 메서드**를 호출
    - 빌트인 데이터의 경우 **스페셜 메서드**를 호출하지 않는 경우도 있으며, 이 같은 경우에는 메서드 호출보다 빠르게 연산이 수행됨
    - 단순히 **struct**에서 **필드 값**을 읽어오는 연산을 수행하면 되기 때문 !
- 현재 사용되지 않는 **스페셜 메서드** 명이 미래에 사용될 수도 있기 때문에 **스페셜 메서드**를 함부로 정의하지 않는 것이 좋음
    - e.g.) `__foo__`와 같이 임의의 **스페셜 메서드**를 정의해 사용하고 있는데 파이썬 **4.x** 와 같은 버전에서 해당 이름을 사용하는 **스페셜 메서드**가 추가될 경우, 프로그램에서 충돌이 발생하게 됨 !

## 1.2.1 숫자 자료형 모방하기
- 2차원 벡터를 표현하는 클래스를 만들며 **스페셜 메서드**를 더 이해해보자

In [13]:
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

`__init__`을 제외하고 5개의 **스페셜 메서드**를 구현했지만, 해당 메서드들은 사용자에 의해 직접 호출되는 메서드들임이 아님

In [14]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
v1 + v2  #v1.__add__(v2)

Vector(4, 5)

`+` 연산의 결과 값으로 `Vector` 클래스가 나왔음

In [15]:
v = Vector(3, 4)
abs(v)   # v.__abs__()

5.0

`abs()` 함수를 활용해 **벡터의 크기**를 구할 수 있음: **L2 Norm**

In [16]:
v * 3  # v.__mul__(3)

Vector(9, 12)

`*` 연산자를 사용해 **Scalar Multiplication**도 구현 가능

### 1.2.2 문자열 표현
- `__repr__` 메서드가 구현되어 있지 않다면 **Vector 객체**는 출력 시 **<Vector object at 주소>**와 같은 형태로 출력되지만, 해당 메서드를 구현함으로써 문자열로 해당 객체에 대한 설명을 출력해줄 수 있음
- 포매팅은 이제 **f-string**을 사용하도록 하자!
- `__repr__` 메서드가 반환하는 문자열은 가능한 해당 객체를 재생성하는데 필요한 소스 코드와 같아야 함: 때문에 **Vector(3, 4)**와 같은 형태로 반환
- `__str__`은 `print()`에 의해 암묵적으로 사용되는 **스페셜 메서드**이며, `__str__`과 `__repr__` 중에 하나만 구현할 것이라면 `__repr__` 함수를 구현
    - 어차피 `__str__` 메서드가 구현되어 있지 않다면 **인터프리터**가 자동으로 `__repr__` 메서드를 호출

### 1.2.3 산술 연산자
- `+`와 `*`와 같은 **중위 연산자**는 각각 `__add__`, `__mul__`으로 정의되며, 해당 연산들은 관습적으로 피연산자를 변경하지 않고 결과 객체를 새로이 반환

### 1.2.4 사용자 정의형의 불리언 값
- `__bool__`이나 `__len__`을 구현하지 않은 경우, 기본적으로 새로이 정의된 클래스의 객체는 **True**로 간주
- `__len__`이 구현되어 있으면 파이썬은 `x.__len__()`을 호출
    - 해당 메서드가 **0**을 반환하면 `bool(x)`은 **False**를,
    - 그렇지 않으면 **True**를 반환
- 위 예제에서는 **벡터의 크기**가 0이면 **False**를, 그렇지 않으면 **True**를 반환하게 하기 위해 `abs(self)`를 활용

In [17]:
class foo:
    def __init__(self):
        pass
    
f = foo()
bool(f)

True

## 1.4 왜 len()은 메서드가 아닐까?
- **CPython**의 내장 객체에 대해서는 메서드를 호출하지 않고 **C 언어 구조체의 필드 값**을 읽어오기 때문에 훨씬 빠름
- 컬렉션의 원소 개수를 가져오는 연산은 빈번하게 발생하기 때문에 `str`, `list`, `memoryview` 등 **기본형 객체**들에 대해 효율적으로 작동해야 함
- 따라서 `len()`은 `abs()`와 같이 **파이썬 데이터 모델**에서 특별 대우를 받으므로 굳이 메서드라고 부르지 않음: 하나의 **연산**과 같이 취급

## 1.5 요약
- **스페셜 메서드**를 구현하면 **사용자 정의 객체**도 내장형 객체처럼 작동하게 되며, 이를 통해 **파이써닉한 코딩**이 가능해짐
- `__repr__`과 `__str__`이라는 **스페셜 메서드**의 존재 이유는 객체 정보를 **디버깅 및 로그**에 사용하고, 사용자가 확인할 수 있도록 하기 위함